# Recurrent Neural Network
###### Graycode using tensorflow

Import dependencies

In [1]:
import numpy as np
import tensorflow as tf

### Pre-process data

In [2]:
# seed random number generators
np.random.seed(1)
tf.set_random_seed(1)

In [3]:
total_test_cases = 100
train_test_ratio = 0.80

tmp_list = []
features = []
labels = []

function to generate graycode of number given bits

In [4]:
def graycode(x, mx):
    return int(bin(mx + x ^ int(x / 2))[3:], 2)

generate data (graycode)

In [5]:
for i in range(total_test_cases):
    a = np.random.randint(0, 256)
    b = graycode(a, 256)

    features.append(a)
    labels.append(b)

features = np.array(features, dtype=np.uint8).reshape(-1, 1)
labels = np.array(labels, dtype=np.uint8).reshape(-1, 1)
features = np.unpackbits(features, axis=1)
labels = np.unpackbits(labels, axis=1)

features = np.expand_dims(features, axis=2)
labels = np.expand_dims(labels, axis=2)

split into train-test set and transpose the array

In [6]:
features_train = np.transpose(np.array(features[:int(train_test_ratio * len(features))]), [1, 0, 2])
features_test = np.transpose(np.array(features[int(train_test_ratio * len(features)):]), [1, 0, 2])

labels_train = np.transpose(labels[:int(train_test_ratio * len(labels))], [1, 0, 2])
labels_test = np.transpose(labels[int(train_test_ratio * len(labels)):], [1, 0, 2])

## Neural Network

hyper-parameters

In [7]:
n_input_neurons = 1
n_rnn_neurons = 8
n_output_neurons = 1
sequence_len = 8

learning_rate = 0.01

n_epochs = 100

input/output placeholders

In [8]:
X = tf.placeholder(tf.float32, [sequence_len, None, n_input_neurons])
Y = tf.placeholder(tf.float32, [sequence_len, None, n_output_neurons])

weights and biases

In [9]:
layer_op = {
    'weight': tf.Variable(tf.random_normal([n_rnn_neurons, n_output_neurons], stddev=0.1)),
    'bias': tf.Variable(tf.random_normal([n_output_neurons], stddev=0.1))
}

#### Model

In [10]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(n_rnn_neurons)
rnn_ops, rnn_states = tf.nn.dynamic_rnn(rnn_cell, X, time_major=True, dtype=tf.float32)

pred_op = tf.map_fn(lambda x: tf.nn.sigmoid(tf.matmul(x, layer_op['weight']) + layer_op['bias']), rnn_ops)

#### Error and Optimizer

In [11]:
# mean-squared error
error = tf.reduce_mean(0.5 * tf.square(pred_op - Y))

# adam-optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(error)

#### Start Session

In [12]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    print("########## TRAIN ##########")

    # Epoch training
    for epoch in range(n_epochs):
        _, err = sess.run([optimizer, error], feed_dict={X: features_train, Y: labels_train})
        
        if epoch % 10 == 0:
            print("Epoch:", epoch, " Error:", err)

    print("\n########## TEST ##########")

    op = pred_op.eval({X: features_test})
    op = (op > 0.5).astype(int)
    
    op = np.packbits(op, 0)[0]
    a = np.packbits(features_test, 0)[0]
    b = np.packbits(labels_test, 0)[0]
    
    success = 0
    
    for i in range(len(op)):
        
        if b[i] == op[i]:
            success += 1
            
        print("%d => %d \t --> %5s " % (a[i], op[i], (b[i] == op[i])[0]))
            
    print("\nSuccess: %d/%d, Accuracy = %f" % (success, len(op), success / len(op) * 100))


########## TRAIN ##########
Epoch: 0  Error: 0.12597
Epoch: 10  Error: 0.121528
Epoch: 20  Error: 0.116221
Epoch: 30  Error: 0.10926
Epoch: 40  Error: 0.0992811
Epoch: 50  Error: 0.08634
Epoch: 60  Error: 0.0719298
Epoch: 70  Error: 0.058572
Epoch: 80  Error: 0.0472783
Epoch: 90  Error: 0.0373565

########## TEST ##########
15 => 8 	 -->  True 
64 => 96 	 -->  True 
196 => 166 	 -->  True 
25 => 21 	 -->  True 
111 => 88 	 -->  True 
226 => 147 	 -->  True 
215 => 188 	 -->  True 
135 => 196 	 -->  True 
26 => 23 	 -->  True 
153 => 213 	 -->  True 
104 => 92 	 -->  True 
22 => 29 	 -->  True 
9 => 13 	 -->  True 
195 => 162 	 -->  True 
231 => 148 	 -->  True 
126 => 65 	 -->  True 
23 => 28 	 -->  True 
125 => 67 	 -->  True 
100 => 86 	 -->  True 
155 => 214 	 -->  True 

Success: 20/20, Accuracy = 100.000000
